In [304]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [333]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
from ydata_profiling import ProfileReport
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, KFold
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import RidgeCV, LassoCV
from sklearn.ensemble import RandomForestRegressor

In [334]:
data = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')

In [335]:
data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


In [336]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

In [337]:
data.columns


Index(['Id', 'MSSubClass', 'MSZoning', 'LotFrontage', 'LotArea', 'Street',
       'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
       'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType',
       'HouseStyle', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd',
       'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType',
       'MasVnrArea', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual',
       'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinSF1',
       'BsmtFinType2', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'Heating',
       'HeatingQC', 'CentralAir', 'Electrical', '1stFlrSF', '2ndFlrSF',
       'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath',
       'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'KitchenQual',
       'TotRmsAbvGrd', 'Functional', 'Fireplaces', 'FireplaceQu', 'GarageType',
       'GarageYrBlt', 'GarageFinish', 'GarageCars', 'GarageArea', 'GarageQual',
       'GarageCond', 'PavedDrive

In [338]:
data = data.drop(['Id','Alley','PoolQC','Fence','MiscFeature'],axis=1)
#결측치가 너무 높은 칼럼 제거

In [339]:
data = data.drop(['GarageCars','TotRmsAbvGrd','1stFlrSF','2ndFlrSF','LowQualFinSF'],axis=1)
#데이터가 비슷한 칼럼 제거

In [340]:
data['TotalBath'] = data['BsmtFullBath']+(0.5*data['BsmtHalfBath'])+data['FullBath']+(0.5*data['HalfBath'])
data = data.drop(['BsmtFullBath','BsmtHalfBath','FullBath','HalfBath'],axis =1 )

In [313]:
# report = ProfileReport(data)
# report

In [341]:
data = data.drop(['BsmtFinSF1','BsmtFinSF2','BsmtUnfSF'],axis =1 )

In [342]:
data['HouseAge']= data['YrSold']-data['YearBuilt']
data['RemodAge']= data['YrSold']-data['YearRemodAdd']
data['GarageAge']= data['YrSold']-data['GarageYrBlt']


In [343]:
data = data.drop(['YrSold','YearBuilt','YearRemodAdd','GarageYrBlt'],axis = 1)

In [344]:
quality_cols = ["ExterQual", "ExterCond", 
    "BsmtQual", "BsmtCond", "GarageQual", "GarageCond", 
    "KitchenQual", "HeatingQC", "FireplaceQu",]

df_quality = data[quality_cols].copy()

data = data.drop(["ExterQual", "ExterCond", 
    "BsmtQual", "BsmtCond", "GarageQual", "GarageCond", 
    "KitchenQual", "HeatingQC", "FireplaceQu"],axis =1)

# 확인
print("메인 데이터 shape:", data.shape)
print("품질 데이터 shape:", df_quality.shape)


메인 데이터 shape: (1460, 55)
품질 데이터 shape: (1460, 9)


In [345]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 55 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  RoofStyle      1460 non-null   object 
 18  RoofMatl

In [346]:
def missing_report(df):
    miss = df.isna().sum()
    pct = miss / len(df) * 100
    rep = pd.DataFrame({'missing': miss, 'pct': pct}).sort_values('pct', ascending=False)
    return rep[rep['missing'] > 0]

missing_report(data)

,missing,pct
MasVnrType,872,59.726027
LotFrontage,259,17.739726
GarageAge,81,5.547945
GarageFinish,81,5.547945
GarageType,81,5.547945
BsmtFinType2,38,2.602740
BsmtExposure,38,2.602740
BsmtFinType1,37,2.534247
MasVnrArea,8,0.547945
Electrical,1,0.068493


In [347]:
numeric_cols = data.select_dtypes(include=[np.number]).columns.tolist()


In [348]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 55 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   MSSubClass     1460 non-null   int64  
 1   MSZoning       1460 non-null   object 
 2   LotFrontage    1201 non-null   float64
 3   LotArea        1460 non-null   int64  
 4   Street         1460 non-null   object 
 5   LotShape       1460 non-null   object 
 6   LandContour    1460 non-null   object 
 7   Utilities      1460 non-null   object 
 8   LotConfig      1460 non-null   object 
 9   LandSlope      1460 non-null   object 
 10  Neighborhood   1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Condition2     1460 non-null   object 
 13  BldgType       1460 non-null   object 
 14  HouseStyle     1460 non-null   object 
 15  OverallQual    1460 non-null   int64  
 16  OverallCond    1460 non-null   int64  
 17  RoofStyle      1460 non-null   object 
 18  RoofMatl

In [349]:
# # data['LotFrontage'].median()
# data['LotFrontage'].unique()
# # LotFrontage 컬럼의 NaN 값을 중앙값으로 채우기
data['LotFrontage'] = data['LotFrontage'].fillna(data['LotFrontage'].median())
# # data['MasVnrArea'] = 
# data['MasVnrArea'].fillna(data['MasVnrArea']).mean()

In [323]:



# def winsorize_iqr(df, numeric_cols, k=1.5):
#     """수치형 데이터에 대해 IQR 기반 윈저라이즈(클리핑)를 수행."""
#     df_w = df.copy()
#     bounds = {}
#     for c in numeric_cols:
#         s = df_w[c].dropna()
#         if s.empty:
#             continue
#         q1, q3 = s.quantile([0.25, 0.75])
#         iqr = q3 - q1
#         if iqr == 0:
#             continue
#         low, high = q1 - k * iqr, q3 + k * iqr
#         before = df_w[c].copy()
#         df_w[c] = df_w[c].clip(lower=low, upper=high)
#         n_clipped = int((before < low).sum() + (before > high).sum())
#         bounds[c] = {'low': low, 'high': high, 'n_clipped': n_clipped}

#     return df_w, pd.DataFrame(bounds).T.sort_values('n_clipped', ascending=False)

In [324]:

# def winsorize_iqr(df, numeric_cols, k=1.5):
#     """수치형 데이터에 대해 IQR 기반 윈저라이즈(클리핑)를 수행."""
#     df_w = df.copy()
#     bounds = {}
#     for c in numeric_cols:
#         s = df_w[c].dropna()
#         if s.empty:
#             continue
#         q1, q3 = s.quantile([0.25, 0.75])
#         iqr = q3 - q1
#         if iqr == 0:
#             continue
#         low, high = q1 - k * iqr, q3 + k * iqr
#         before = df_w[c].copy()
#         df_w[c] = df_w[c].clip(lower=low, upper=high)
#         n_clipped = int((before < low).sum() + (before > high).sum())
#         bounds[c] = {'low': low, 'high': high, 'n_clipped': n_clipped}

#     return df_w, pd.DataFrame(bounds).T.sort_values('n_clipped', ascending=False)

# def process_data(df):
#     """수치형과 범주형 데이터를 처리하는 함수"""
#     # 수치형 컬럼 찾기
#     numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
    
#     # 범주형 컬럼 찾기
#     categorical_cols = df.select_dtypes(exclude=[np.number]).columns.tolist()

#     # 수치형 데이터에 대해 결측치 처리 및 이상치 클리핑 수행
#     # 결측치는 중간값으로 채움
#     numeric_pipeline = Pipeline(steps=[
#         ('imputer', SimpleImputer(strategy='median')),  # 결측치 처리
#         ('scaler', StandardScaler())  # 정규화
#     ])

#     # 범주형 데이터에 대해 결측치 처리 및 원핫인코딩 수행
#     categorical_pipeline = Pipeline(steps=[
#         ('imputer', SimpleImputer(strategy='most_frequent')),  # 결측치 처리
#         ('onehot', OneHotEncoder(handle_unknown='ignore'))  # 원핫 인코딩
#     ])

#     # 수치형 데이터는 윈저라이즈(IQR 클리핑) 적용
#     df, clip_report = winsorize_iqr(df, numeric_cols, k=1.5)
    
#     # 전처리 파이프라인 적용
#     preprocessor = ColumnTransformer(
#         transformers=[
#             ('num', numeric_pipeline, numeric_cols),  # 수치형 파이프라인
#             ('cat', categorical_pipeline, categorical_cols)  # 범주형 파이프라인
#         ]
#     )
    
#     # 변환된 데이터 반환
#     X = preprocessor.fit_transform(df)
    
#     # 윈저라이즈 클리핑 리포트 출력
#     print("IQR 윈저라이즈 클리핑 리포트:")
#     print(clip_report)
    
#     return X, df

# # 예시 데이터프레임
# df = pd.DataFrame({
#     'Age': [25, 30, 35, 1000, 40],  # 수치형
#     'Salary': [50000, 55000, 60000, 70000, 80000],  # 수치형
#     'Gender': ['Male', 'Female', 'Female', 'Male', np.nan],  # 범주형
#     'City': ['New York', 'Los Angeles', 'Chicago', 'New York', 'Chicago']  # 범주형
# })

# # 데이터 처리
# X, processed_df = process_data(df)

# # 처리된 데이터 확인
# print("\n처리된 데이터:")
# print(X)


In [325]:

# # ===== 1) 유틸 함수 =====
# def missing_report(df):
#     miss = df.isna().sum()
#     pct = miss / len(df) * 100
#     rep = pd.DataFrame({'missing': miss, 'pct': pct}).sort_values('pct', ascending=False)
#     return rep[rep['missing'] > 0]

# def winsorize_iqr(df, numeric_cols, k=1.5, exclude=None):
#     """IQR 기반 윈저라이즈(클리핑). exclude에 넣은 컬럼은 건너뜀."""
#     if exclude is None:
#         exclude = []
#     df_w = df.copy()
#     bounds = {}
#     for c in numeric_cols:
#         if c in exclude:
#             continue
#         s = df_w[c].dropna()
#         if s.empty:
#             continue
#         q1, q3 = s.quantile([0.25, 0.75])
#         iqr = q3 - q1
#         if iqr == 0:
#             continue
#         low, high = q1 - k*iqr, q3 + k*iqr
#         before = df_w[c].copy()
#         df_w[c] = df_w[c].clip(lower=low, upper=high)
#         n_clipped = int((before < low).sum() + (before > high).sum())
#         bounds[c] = {'low': low, 'high': high, 'n_clipped': n_clipped}
#     return df_w, pd.DataFrame(bounds).T.sort_values('n_clipped', ascending=False)

# def mode(series):
#     return series.mode().iloc[0] if not series.mode().empty else np.nan

# # ===== 2) 도메인 규칙 기반 클리너 =====
# def clean_ames(df, target_col="SalePrice"):
#     dfc = df.copy()

#     # 오타 컬럼 정리: GatageAge -> GarageAge
#     if 'GatageAge' in dfc.columns and 'GarageAge' not in dfc.columns:
#         dfc = dfc.rename(columns={'GatageAge': 'GarageAge'})

#     # 결측 보고서(원본 기준)
#     report_before = missing_report(dfc)

#     # ---- 범주형 결측: "None" 또는 최빈값 ----
#     fill_none_cols = [
#         # 지하, 차고, 난방 품질/타입 계열(현재 남아있는 것들만 적용됨)
#         'BsmtExposure','BsmtFinType1','BsmtFinType2',
#         'GarageType','GarageFinish'
#     ]
#     for c in fill_none_cols:
#         if c in dfc.columns:
#             dfc[c] = dfc[c].fillna("None")

#     # MasVnrType/Area 관례
#     if 'MasVnrType' in dfc.columns:
#         dfc['MasVnrType'] = dfc['MasVnrType'].fillna("None")
#     if 'MasVnrArea' in dfc.columns:
#         dfc['MasVnrArea'] = dfc['MasVnrArea'].fillna(0)

#     # Electrical 1개 결측(보통 모드로)
#     if 'Electrical' in dfc.columns and dfc['Electrical'].isna().any():
#         dfc['Electrical'] = dfc['Electrical'].fillna(mode(dfc['Electrical']))

#     # ---- LotFrontage: Neighborhood별 중앙값으로 보간 + 결측 플래그 ----
#     if 'LotFrontage' in dfc.columns:
#         dfc['LotFrontage_missing'] = dfc['LotFrontage'].isna().astype(int)
#         if 'Neighborhood' in dfc.columns:
#             med_by_nbhd = dfc.groupby('Neighborhood')['LotFrontage'].transform(lambda s: s.fillna(s.median()))
#             dfc['LotFrontage'] = dfc['LotFrontage'].fillna(med_by_nbhd)
#         # 남은 결측이 있으면 전체 중앙값으로
#         if dfc['LotFrontage'].isna().any():
#             dfc['LotFrontage'] = dfc['LotFrontage'].fillna(dfc['LotFrontage'].median())

#     # ---- GarageAge: 결측 플래그 + 0으로 채움(차고 없음의 의미)
#     if 'GarageAge' in dfc.columns:
#         dfc['GarageAge_missing'] = dfc['GarageAge'].isna().astype(int)
#         dfc['GarageAge'] = dfc['GarageAge'].fillna(0)

#     # ---- 나머지 범주형: 최빈값으로 ----
#     cat_cols = dfc.select_dtypes(include=['object']).columns.tolist()
#     for c in cat_cols:
#         dfc[c] = dfc[c].fillna(mode(dfc[c]))

#     # ---- 수치형: 여기서는 일단 그대로 두고, 파이프라인에서 median impute + indicator 처리 ----
#     num_cols = dfc.select_dtypes(include=[np.number]).columns.tolist()
#     # target은 outlier clip에서 제외
#     if target_col in num_cols:
#         num_cols.remove(target_col)

#     # ---- IQR 윈저라이즈 (0이 구조적 의미인 칼럼은 제외 권장) ----
#     exclude_for_clip = ['MoSold']  # 월은 계절성이라 그대로 두는 경우가 많음
#     dfc, clip_report = winsorize_iqr(dfc, numeric_cols=num_cols, k=1.5, exclude=exclude_for_clip)

#     report_after = missing_report(dfc)
#     info = {
#         'missing_before': report_before,
#         'missing_after': report_after,
#         'winsorize_report': clip_report
#     }
#     return dfc, info

# # ===== 3) 전처리 + 모델 파이프라인 =====
# def build_pipelines(df, target_col="SalePrice"):
#     X = df.drop(columns=[target_col])
#     y = df[target_col]

#     numeric_features = X.select_dtypes(include=[np.number]).columns.tolist()
#     categorical_features = X.select_dtypes(include=['object']).columns.tolist()

#     numeric_pipe = Pipeline(steps=[
#         ("imputer", SimpleImputer(strategy="median", add_indicator=True)),
#         ("scaler", StandardScaler())
#     ])

#     categorical_pipe = Pipeline(steps=[
#         ("imputer", SimpleImputer(strategy="most_frequent")),
#         ("ohe", OneHotEncoder(handle_unknown="ignore"))
#     ])

#     preprocessor = ColumnTransformer(
#         transformers=[
#             ("num", numeric_pipe, numeric_features),
#             ("cat", categorical_pipe, categorical_features),
#         ],
#         remainder="drop"
#     )

#     ridge = Pipeline(steps=[
#         ("pre", preprocessor),
#         ("model", RidgeCV(alphas=np.logspace(-3, 3, 25), cv=5))
#     ])

#     lasso = Pipeline(steps=[
#         ("pre", preprocessor),
#         ("model", LassoCV(alphas=np.logspace(-3, 1, 30), cv=5, max_iter=20000))
#     ])

#     rf = Pipeline(steps=[
#         ("pre", preprocessor),
#         ("model", RandomForestRegressor(
#             n_estimators=600, max_depth=None, random_state=42, n_jobs=-1))
#     ])

#     return {"Ridge": ridge, "Lasso": lasso, "RandomForest": rf}

# def rmse_cv(model, X, y, n_splits=5, random_state=42):
#     kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
#     scores = cross_val_score(model, X, y,
#                              scoring="neg_root_mean_squared_error",
#                              cv=kf, n_jobs=-1)
#     return -scores  # 양수 RMSE들

# # ===== 4) 실행 예시 =====
# # df = ...  # 사용중인 DataFrame (SalePrice 포함)
# df_clean, info = clean_ames(df, target_col="SalePrice")

# print("=== 결측치 변화(상위 10행) ===")
# print(info['missing_before'].head(10))
# print("\n=== 정리 후 결측치(상위 10행) ===")
# print(info['missing_after'].head(10))
# print("\n=== IQR 클리핑 리포트(클리핑된 개수 순) ===")
# print(info['winsorize_report'].head(10))

# # 파이프라인 구성 & CV 성능 비교
# pipelines = build_pipelines(df_clean, target_col="SalePrice")
# X = df_clean.drop(columns=["SalePrice"])
# y = df_clean["SalePrice"]

# results = {}
# for name, pipe in pipelines.items():
#     scores = rmse_cv(pipe, X, y, n_splits=5)
#     results[name] = {
#         "RMSE_mean": scores.mean(),
#         "RMSE_std": scores.std(),
#         "fold_RMSEs": np.round(scores, 2)
#     }

# print("\n=== 모델별 5-Fold RMSE ===")
# for k, v in results.items():
#     print(f"{k:12s} -> mean {v['RMSE_mean']:.0f}  (std {v['RMSE_std']:.0f})  folds {list(v['fold_RMSEs'])}")

# # 최종 모델 적합(예: RMSE가 가장 낮은 모델 선택 후 학습)
# best_name = min(results, key=lambda k: results[k]["RMSE_mean"])
# best_model = pipelines[best_name].fit(X, y)
# print(f"\n[선택] 최종 모델: {best_name}")


In [350]:
X = df_main.drop("SalePrice", axis=1)
y = df_main["SalePrice"]

In [351]:
X = pd.get_dummies(X, drop_first=True)

In [352]:
# 결측치가 있는 열만 필터링해서 출력
missing_cols = X.isna().sum()
missing_cols = missing_cols[missing_cols > 0]  # 결측치가 있는 열만 필터링
print(missing_cols)


LotFrontage    259
MasVnrArea       8
GatageAge       81
dtype: int64


In [353]:
X = X.drop(['LotFrontage'],axis = 1)
X = X.drop(['MasVnrArea'],axis = 1)
X = X.drop(['GatageAge'],axis = 1)


In [354]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [355]:
X_train.isnull().sum()

MSSubClass               0
LotArea                  0
OverallQual              0
OverallCond              0
TotalBsmtSF              0
                        ..
SaleCondition_AdjLand    0
SaleCondition_Alloca     0
SaleCondition_Family     0
SaleCondition_Normal     0
SaleCondition_Partial    0
Length: 188, dtype: int64

In [356]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [358]:
y_pred = model.predict(X_test)

In [359]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Linear Regression RMSE:", rmse)

Linear Regression RMSE: 61664.284403649006


In [360]:
from sklearn.metrics import r2_score, mean_absolute_percentage_error

r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test, y_pred)

print(f"R²: {r2:.3f}")
print(f"MAPE: {mape*100:.1f}%")


R²: 0.406
MAPE: 15.8%
